<div>
     <div>
        <img src="./report/isel_logo.png" width="400" height="400" align="left">
    </div>
    <div>
        <h2>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h2>
        <p>Trabalho prático 4</p>
        <p>Autor:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>27 - Junho - 2021</p>
    </div>
</div>

### <a id="index"></a>

# Index
- [Tabelas da norma ITU T.81 - JPEG standard](#ITU_T.81)
- [Compressão](#compressao)
    - [snr](#snr)
- [Entropia](#entropia)
    - [entropy](#entropy)
    - [efficiency](#efficiency)
- [Utilities](#utilities)
    - [imshow](#imshow)
    - [convertFrame2Jpeg](#convertFrame2Jpeg)
    - [createPframe](#createPframe)
    - [mae](#mae)
    - [divide_to_blocks](#divide_to_blocks)
    - [merge_from_blocks](#merge_from_blocks)
- [Codificador 1](#codificador_1)
    - [encode_1](#encode_1)
- [Codificador 2](#codificador_2)
    - [encode_2](#encode_2)
    - [decode_2](#decode_2)
- [Codificador 3](#codificador_2)
    - [encode_3](#encode_3)
    - [decode_3](#decode_3)
- [Testes](#testes)
- [Conclusões](#conclusoes)

# Importar bibliotecas

In [11]:
import os
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd() # current work diretory

<a id="compressao"></a>

# Compressão

A taxa de compressão pode ser cáculada atrás da seguinte expressão matemática:

$$ T_c = \frac{D_o}{D_c} $$

<a id="snr"></a>

## SNR

A relação sinal-ruído compara o nível de um sinal desejado com o nível do ruído. Quanto mais alta for a relação sinal-ruído, menor é o efeito do ruído de fundo sobre a detecção ou medição do sinal.

$$ SNR(Db) = 10\log_{10}\left[\frac{\sum_{l}\sum_{c}I_{ap}(l,c)^2}{\sum_{l}\sum_{c}[I_{ap}(l,c)-I_{or}(l,c)]^2}\right] $$

In [20]:
def snr(I_or, I_ap):
    Pxa = np.sum(I_ap.astype('float')**2)
    Pe = np.sum( (I_ap.astype('float') - I_or.astype('float'))**2 )
    return 10 * np.log10(Pxa / Pe)

<a id="entropia"></a>

# Entropia

A entropia mede a quantidade de informação codificada na mensagem, onde quando maior for o valor entrópico, maior será a incerteza.

A entropia da fonte é dada pela seguinte expressão matemática:

$$ H(S) = -\sum_{i=1}^{N}p(s_i)log_{2}p(s_i) $$


A eficiência da condificação pode ser obtida através da seguinte expressão matemática:

$$ \eta = \frac{H(S)}{L} $$

onde, **_L_** é o número médio de bits por símbolo.

É possível codificar uma fonte, sem perdas, se o número médio de bits por símbolo for maior ou igual à entropia da fonte:

$$ H(S) < L < H(S) + \delta $$

<a id="entropy"></a>

## entropy


In [21]:
def calculate_entropy(symbol_freq_dictionary):
    # list of symbol occurences in file
    occurrences = list(symbol_freq_dictionary.values())
    # total number of symbols
    t = np.sum(occurrences)
    # probability of each symbol
    p = [occ / t for occ in occurrences]
    return -np.sum([p * np.log2(p)])

<a id="efficiency"></a>

## efficiency

In [ ]:
def efficiency(H, dictionary):
    # average number of bits per symbol
    L = 0
    for symbol in dictionary:
        L += len(dictionary[symbol])
    L = L / len(dictionary)
    return L, ( H / L )

[back to index](#index)

<a id="utilities"></a>

# Utilities

<a id="imshow"></a>

## imshow

In [1]:
def imshow(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

<a id="convertFrame2Jpeg"></a>

## convertFrame2Jpeg

In [2]:
def convertFrame2Jpeg(path, frame):
    cv2.imwrite(f"{path}.jpg", frame, (cv2.IMWRITE_JPEG_QUALITY, 50))
    return cv2.imread(f"{path}.jpg", cv2.IMREAD_GRAYSCALE)

<a id="create_Pframe"></a>

## create_Pframe

In [21]:
def create_Pframe(Iframe, frame):
    np.zeros
    return cv2.subtract(Iframe, frame)

In [22]:
Iframe = frames[1][0]
PFrames = [ ]
for i in range(1, len(frames)):
    PFrames.append( createPframe(Iframe, frames[1][i]) )

<a id="reconstruct_Iframe"></a>

## reconstruct_Iframe

In [33]:
def reconstruct_Iframe(Iframe, Pframe):
    return cv2.add(Iframe, Pframe)

<a id="mae"></a>

## Erro absoluto médio (MAE)

Critério de semelhança entre blocos

$$ (d_m,d_n) = arg \min_{d_m,d_n} \frac{1}{MN}\sum_{m=1}^{M}\sum_{n=1}^{N} \left| X^{t}(m,n)-X^{t-1}(m-d_m,n-d_n) \right| $$

In [37]:
def mae(b1, b2):
    return np.sum(np.abs(cv2.subtract(b1,b2).ravel())) / len(b1.ravel())

In [38]:
block1 = np.ones(256).reshape((16,16))
block2 = (np.ones(256)*3).reshape((16,16))

print(mae(block1, block2))

2.0


<a id="divide_to_blocks"></a>

### divide_to_blocks

Divide uma imagem em array de blocos de tamnho size x size.

In [36]:
def divide_to_blocks(image, size):
    output = []
    
    width, height = image.shape
    for lin in range(0, width, size):
        for col in range(0, height, size):
            img_w = lin + size
            img_h = col + size
            blc_w = size
            blc_h = size
            # check for out of bounds
            if (lin + size) > width:
                img_w = width
                blc_w = width - lin
            if (col + size) > height:
                img_h = height
                blc_h = height - col
            # create new block
            block = np.zeros(size*size, dtype='uint8').reshape((size,size))
            block[0:blc_w,0:blc_h] = image[lin:img_w,col:img_h]
            output.append(block)
                
    return output

<a id="merge_from_blocks"></a>

### merge_from_blocks

Merge um array de blocos para uma imagem.

In [3]:
def merge_from_blocks(arr, shape, size):
    width = nearest_multiple(shape[0], size)
    heigth = nearest_multiple(shape[1], size)
    index = 0
    output = np.zeros((width, heigth), dtype='uint8')
    for lin in range(0, width, size):
        for col in range(0, heigth, size):
            output[lin:(lin+size),col:(col+size)] = arr[index]
            index = index + 1
    return output[0:shape[0],0:shape[1]]

 <a id="codificador_1"></a>
 
# Codificador 1
 
<div class="alert alert-block alert-info">
Considera que cada frame é uma intra-frame (I).
</div>

In [7]:
def encode_1(path, frames):
    Iframes = []
    for i in range(len(frames)):
        Iframes.append( convertFrame2Jpeg(f"{path}/Iframe_{i+1}", frames[i]) )
    return Iframes

<a id="codificador_2"></a>
 
# Codificador 2
 
<div class="alert alert-block alert-info">
Considera que todas as frames à exceção da primeira (a I-frame) são inter-frames (P), sem compensação de movimento.
</div>

In [25]:
def encode_2(path, frames):
    Iframe = frames[0]
    Pframes = []
    convertFrame2Jpeg(f"{path}/Iframe_{1}", Iframe)
    for i in range(1, len(frames)):
        Pframes.append( convertFrame2Jpeg(f"{path}/Pframe_{i+1}", create_Pframe(frames[0], frames[i])) )
    return Iframe, Pframes

In [37]:
def decode_2(path, Iframe, Pframes):
    rIframes = []
    for i in range(len(Pframes)):
        rIframes.append( convertFrame2Jpeg(f"{path}/rIframe_{i+2}", reconstruct_Iframe(Iframe, Pframes[i])) )
    return Iframe, rIframes

<a id="codificador_3"></a>
 
# Codificador 3
 
<div class="alert alert-block alert-info">
Considera que todas as frames à exceção da primeira (a I-frame) são inter-frames (P), com compensação de movimento.
</div>

In [ ]:
def encode_3(path, frames):

In [ ]:
def decode_3(path, frames):

### <a id="testes"></a>

# Testes

## Importar dados

In [61]:
# sorting frames names by alphabetical order
# frames[0] = sorted(frames[0], key=lambda x: (x[0],int(x[5:])) )

In [13]:
frames = []
for i in range(len(os.listdir(f"{cwd}/input_data/bola_seq"))):
    frames.append( cv2.imread(f"{cwd}/input_data/bola_seq/bola_{i+1}.tiff", cv2.IMREAD_GRAYSCALE) )

<div class="alert alert-block alert-info">
Codificador 1
</div>

In [28]:
Iframe1 = encode_1(f"{cwd}/output_data/codificador1", frames)

<div class="alert alert-block alert-info">
Codificador 2
</div>

In [38]:
Iframe2, Pframes2 = encode_2(f"{cwd}/output_data/codificador2/encode", frames)

In [39]:
decode_2(f"{cwd}/output_data/codificador2/decode", Iframe2, Pframes2)

(array([[ 60,  62,  62, ...,  30,  32,  32],
        [ 68,  69,  75, ...,  31,  33,  32],
        [ 89,  57,  58, ...,  30,  33,  32],
        ...,
        [ 62,  63,  62, ...,  62,  55,  28],
        [ 62,  64,  65, ..., 120, 119,  99],
        [ 65,  63,  64, ..., 119, 119, 119]], dtype=uint8),
 [array([[ 57,  61,  68, ...,  31,  30,  30],
         [ 69,  68,  67, ...,  31,  31,  30],
         [ 80,  75,  67, ...,  32,  31,  31],
         ...,
         [ 63,  63,  63, ...,  67,  52,  37],
         [ 64,  64,  64, ..., 126, 120, 108],
         [ 64,  64,  64, ..., 124, 118, 102]], dtype=uint8),
  array([[ 57,  61,  68, ...,  33,  32,  32],
         [ 69,  68,  67, ...,  33,  33,  32],
         [ 80,  75,  67, ...,  34,  33,  33],
         ...,
         [ 63,  63,  63, ...,  67,  52,  37],
         [ 64,  64,  64, ..., 126, 120, 108],
         [ 64,  64,  64, ..., 124, 118, 102]], dtype=uint8),
  array([[ 57,  61,  68, ...,  33,  32,  32],
         [ 69,  68,  67, ...,  33,  33,  32],


<div class="alert alert-block alert-info">
Codificador 3
</div>

[back to index](#index)

<a id="conclusoes"></a>

# Conclusões

Este trabalho explora os princípios básicos da codificação de vídeo. Neste trabalho pretende-se implementar três formas
de codificação de vídeo. Cada um destes codificadores deve ser testado com a a sequência de imagens disponibilizadas

[back to index](#index)